In [1]:
!pip install pandas
!pip install pymongo
!pip install boto3

In [3]:
import numpy as np
import pandas as pd

In [15]:
from io import BytesIO
from pymongo import MongoClient
import boto3
from io import StringIO
import shutil
import csv

In [11]:
client = boto3.client('s3', 
    endpoint_url='http://awari-minio-nginx:9000',
    aws_access_key_id='l1Bl6oNPKVVVhPMN',
    aws_secret_access_key='N1n2kHSzUFqs38T7TIPmXxJgusRZcbfc',
    aws_session_token=None,
    config=boto3.session.Config(signature_version='s3v4'),
    verify=False,
    region_name='sa-east-1'
)

In [ ]:
# Definindo os arquivos. 
arquivos =  os.listdir('./exercicios/municipios-estados/streaming')
arquivos = [arquivo for arquivo in arquivos if arquivo.endswith('.csv')]
arquivos

for i in arquivos:
    # Lendo o arquivo CSV
    df = pd.read_csv(f'./exercicios/municipios-estados/streaming/{i}')
    
    # Supondo que 'UF' seja a coluna com a Unidade Federativa e pegando o primeiro valor
    uf = df['codigo_uf'].iloc[0]
    
    # Criando a subpasta para a UF, se não existir
    if not os.path.exists(f"./exercicios/municipios-estados/streaming/{codigo_uf}"):
        os.makedirs(f"./exercicios/municipios-estados/streaming/{codigo_uf}")
    
    # Salvando o arquivo no bucket do Minio
    csv_buffer = StringIO()
    csv = df.to_csv(csv_buffer, index=False)
    client.put_object(Body=csv_buffer.getvalue(), Bucket='aula-07', Key=f'{codigo_uf}/{i}')
    
    # Adicionando o conteúdo do arquivo ao final de cidades.csv na subpasta da UF
    with open(f"./exercicios/municipios-estados/streaming/{codigo_uf}/cidades.csv", 'a') as f:
        df.to_csv(f, header=f.tell()==0, index=False)  # Só adicionar header se for o início do arquivo
    
    # Movendo o arquivo para a pasta tratados
    shutil.move(f"./exercicios/municipios-estados/streaming/{i}", "./arquivos/tratados")

